## NLP for PyTorch

Maintenant que nous avons appris à construire des réseaux de neurones, nous allons voir comment il est possible de construire des modèles pour le NLP à l'aide de PyTorch. Dans cet exemple, nous allons créer un classificateur de sacs de mots de base afin de classer la langue d'une phrase donnée

## Mise en place de modèle de classificatin 

Pour cet exemple, nous allons prendre une sélection de phrases en espagnol et en anglais :

1. Tout d'abord, nous divisons chaque phrase en une liste de mots et prenons la langue de chaque phrase comme étiquette. Nous prenons une section de phrases sur laquelle entraîner notre modèle et gardons une petite section de côté comme ensemble de test. Nous faisons cela pour pouvoir évaluer les performances de notre modèle après son entraînement :

In [2]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch import nn, optim

In [3]:
training_data = [
        ("Veinte paginas".lower().split(), "Spanish"),
        ("I will visit the library".lower().split(), "English"),
        ("I am reading a book".lower().split(), "English"),
        ("This is my favourite chapter".lower().split(), "English"),
        ("Estoy en la biblioteca".lower().split(), "Spanish"),
        ("Tengo un libro".lower().split(), "Spanish")
        ]

test_data = [
        ("Estoy leyendo".lower().split(), "Spanish"),
        ("This is not my favourite book".lower().split(), "English")
        ]

Notez que nous transformons également chaque mot en minuscule, ce qui empêche les mots d'être comptés deux fois dans notre sac de mots. Si nous avons le mot LIVRE et le mot livre, nous voulons qu'ils soient comptés comme le même mot, donc nous les transformons en minuscules

2. Ensuite, nous construisons notre index de mots, qui est simplement un dictionnaire de tous les mots de notre corpus, puis créons une valeur d'index unique pour chaque mot. Cela peut être facilement fait avec une courte boucle for :

In [4]:
word_dict = {}
i = 0
for words, language in training_data + test_data:
    for word in words:
        if word not in word_dict:
            word_dict[word] = i
            i += 1
print(word_dict)

{'veinte': 0, 'paginas': 1, 'i': 2, 'will': 3, 'visit': 4, 'the': 5, 'library': 6, 'am': 7, 'reading': 8, 'a': 9, 'book': 10, 'this': 11, 'is': 12, 'my': 13, 'favourite': 14, 'chapter': 15, 'estoy': 16, 'en': 17, 'la': 18, 'biblioteca': 19, 'tengo': 20, 'un': 21, 'libro': 22, 'leyendo': 23, 'not': 24}


Notez qu'ici, nous avons parcouru toutes nos données d'entraînement et nos données de test. Si nous créions simplement notre index de mots sur les données d'entraînement, lorsqu'il s'agirait d'évaluer notre ensemble de tests, nous aurions de nouveaux mots qui n'étaient pas vus dans l'entraînement d'origine, nous ne serions donc pas en mesure de créer un véritable sac de mots représentation de ces mots.


3. Maintenant, nous construisons notre classificateur de la même manière que nous avons construit notre réseau de neurones dans la section précédente ; c'est-à-dire en construisant une nouvelle classe qui hérite de nn.Module.


Ici, nous définissons notre classificateur de sorte qu'il se compose d'une seule couche linéaire avec des fonctions d'activation log somax approchant une régression logistique. Nous pourrions facilement étendre cela pour fonctionner comme un réseau de neurones en ajoutant ici des couches linéaires supplémentaires, mais une seule couche de paramètres servira notre objectif. Portez une attention particulière aux tailles d'entrée et de sortie de notre couche linéaire :

In [5]:
corpus_size = len(word_dict)
languages = 2
label_index = {"Spanish": 0, "English": 1}



class BagofWordsClassifier(nn.Module):  

    def __init__(self, languages, corpus_size):
        super(BagofWordsClassifier, self).__init__()
        self.linear = nn.Linear(corpus_size, languages)

    def forward(self, bow_vec):
        return F.log_softmax(self.linear(bow_vec), dim=1)

L'entrée est de longueur corpus_size, qui est juste le nombre total de mots uniques dans notre corpus. Cela est dû au fait que chaque entrée de notre modèle sera une représentation de sac de mots, composée du nombre de mots dans chaque phrase, avec un nombre de 0 si un mot donné n'apparaît pas dans notre phrase. Notre sortie est de taille 2, qui est notre nombre de langues à prédire. Nos prédictions finales consisteront en une probabilité que notre phrase soit en anglais par rapport à la probabilité que notre phrase soit en espagnol, notre prédiction finale étant celle avec la probabilité la plus élevée.


4. Ensuite, nous définissons quelques fonctions utilitaires. Nous définissons d'abord make_bow_vector, qui prend la phrase et la transforme en une représentation de sac de mots. Nous créons d'abord un vecteur composé de tous les zéros. Nous les parcourons ensuite et pour chaque mot de la phrase, nous incrémentons de un le nombre de cet index dans le vecteur du sac de mots. Nous remodelons finalement ce vecteur en utilisant avec .view() pour l'entrée dans notre classificateur :

In [6]:

def make_bow_vector(sentence, word_index):
    word_vec = torch.zeros(corpus_size)
    for word in sentence:
        word_vec[word_dict[word]] += 1
    return word_vec.view(1, -1)

5. De même, nous définissons make_target, qui prend simplement l'étiquette de la phrase (espagnol ou anglais) et renvoie son index pertinent (0 ou 1) :

In [7]:
def make_target(label, label_index):
    return torch.LongTensor([label_index[label]])

6. Nous pouvons maintenant créer une instance de notre modèle, prête pour l'entraînement. Nous définissons également notre fonction de perte en tant que probabilité de log négatif car nous utilisons une fonction log somax, puis définissons notre optimiseur afin d'utiliser la descente de gradient stochastique standard (SGD)

In [8]:
model = BagofWordsClassifier(languages, corpus_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

## Former le modèle
Tout d'abord, nous mettons en place une boucle constituée du nombre d'époques pour lesquelles nous souhaitons que notre modèle s'exécute. Dans ce cas, nous sélectionnerons 50 époques.

Dans cette boucle, nous mettons d'abord nos gradients à zéro (autrement, PyTorch calcule les gradients de manière cumulative), puis pour chaque paire phrase/étiquette, nous transformons chacun en un vecteur et une cible de sac de mots, respectivement. Nous calculons ensuite la sortie prédite de cette paire de phrases particulière en effectuant un passage en avant de nos données à travers l'état actuel de notre modèle.

En utilisant cette prédiction, nous prenons ensuite nos étiquettes prédites et réelles et appelons notre fonction loss_définie sur les deux pour obtenir une mesure de la perte pour cette phrase. En appelant backward(), nous rétropropageons ensuite cette perte à travers notre modèle et en appelant step() sur notre optimiseur, nous mettons à jour nos paramètres de modèle. Enfin, nous imprimons notre perte après toutes les 10 étapes d'entraînement

In [9]:
for epoch in range(100):
    for sentence, label in training_data:

        model.zero_grad()

        bow_vec = make_bow_vector(sentence, word_dict)
        target = make_target(label, label_index)

        log_probs = model(bow_vec)

        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        
    if epoch % 10 == 0:
        print('Epoch: ',str(epoch+1),', Loss: ' + str(loss.item()))

Epoch:  1 , Loss: 1.0349431037902832
Epoch:  11 , Loss: 0.15093868970870972
Epoch:  21 , Loss: 0.07269478589296341
Epoch:  31 , Loss: 0.0473831407725811
Epoch:  41 , Loss: 0.03504699096083641
Epoch:  51 , Loss: 0.027775410562753677
Epoch:  61 , Loss: 0.022989528253674507
Epoch:  71 , Loss: 0.019604023545980453
Epoch:  81 , Loss: 0.01708410121500492
Epoch:  91 , Loss: 0.015136264264583588


Ici, nous pouvons voir que notre perte diminue au fil du temps à mesure que notre modèle apprend. Bien que notre ensemble de formation dans cet exemple soit très petit, nous pouvons toujours démontrer que notre modèle a appris quelque chose d'utile, comme suit :

1. Nous évaluons notre modèle sur quelques phrases de nos données de test sur lesquelles notre modèle n'a pas été formé. Ici, nous avons d'abord défini torch.no_grad(), qui désactive le moteur d'autograd car il n'est plus nécessaire de calculer les gradients car nous n'entraînons plus notre modèle. Ensuite, nous prenons notre phrase de test et la transformons en un vecteur de sac de mots et l'injectons dans notre modèle pour obtenir des prédictions.

2. Nous imprimons ensuite simplement la phrase, la véritable étiquette de la phrase, puis les probabilités prédites. Notez que nous retransformons les valeurs prédites des probabilités de log en probabilités. Nous obtenons deux probabilités pour chaque prédiction, mais si nous nous référons à l'indice d'étiquette, nous pouvons voir que la première probabilité (indice 0) correspond à l'espagnol, tandis que l'autre correspond à l'anglais

In [10]:
def make_predictions(data):

    with torch.no_grad():
        sentence = data[0]
        label = data[1]
        bow_vec = make_bow_vector(sentence, word_dict)
        log_probs = model(bow_vec)
        print(sentence)
        print(label + ':')
        print(np.exp(log_probs))

make_predictions(test_data[0])
make_predictions(test_data[1])

['estoy', 'leyendo']
Spanish:
tensor([[0.8411, 0.1589]])
['this', 'is', 'not', 'my', 'favourite', 'book']
English:
tensor([[0.0141, 0.9859]])


Ici, nous pouvons voir que pour nos deux prédictions, notre modèle prédit la bonne réponse, mais pourquoi est-ce ? Qu'est-ce que notre modèle a appris exactement ? Nous pouvons voir que notre première phrase de test contient le mot estoy, qui a déjà été vu dans une phrase espagnole au sein de notre ensemble d'apprentissage. De même, nous pouvons voir que le mot book a été vu dans notre formation dans une phrase anglaise. Puisque notre modèle se compose d'une seule couche, les paramètres sur chacun de nos nœuds sont faciles à interpréter. 

3. Ici, nous définissons une fonction qui prend un mot en entrée et renvoie les poids sur chacun des paramètres de la couche. Pour un mot donné, nous récupérons l'index de ce mot dans notre dictionnaire puis sélectionnons ces paramètres à partir du même index au sein du modèle. Notez que notre modèle renvoie deux paramètres car nous faisons deux prédictions ; c'est-à-dire la contribution du modèle à la prédiction espagnole et la contribution du modèle à la prédiction anglaise :

In [12]:

def return_params(word): 
    index = word_dict[word]
    for p in model.parameters():
        dims = len(p.size())
        if dims == 2:
            print(word + ':')
            print('Spanish Parameter = ' + str(p[0][index].item()))
            print('English Parameter = ' + str(p[1][index].item()))
            print('\n')
            
return_params('estoy')
return_params('book')

estoy:
Spanish Parameter = 0.46073251962661743
English Parameter = -0.5677701234817505


book:
Spanish Parameter = -0.398368239402771
English Parameter = 0.5698005557060242




Ici, nous pouvons voir que pour le mot estoy, ce paramètre est positif pour la prédiction espagnole et négatif pour la prédiction anglaise. Cela signifie que pour chaque compte du mot "estoy" dans notre phrase, la phrase devient plus susceptible d'être une phrase espagnole. De même, pour le mot livre, nous pouvons voir que cela contribue positivement à la prédiction que la phrase est en anglais. Nous pouvons montrer que notre modèle n'a appris qu'en fonction de ce sur quoi il a été formé. Si nous essayons de prédire un mot sur lequel le modèle n'a pas été entraîné, nous pouvons voir qu'il est incapable de prendre une décision précise. Dans ce cas, notre modèle pense que le mot anglais "not" est espagnol
